In [ ]:
# tracker.py
# Author: <Your Name>
# Date: 2025-11-10
# Assignment: Command-Line Attendance Tracker
#
# Purpose: Simple attendance tracker that collects student names and check-in times,
# validates input, prevents duplicates, shows a formatted summary, optionally calculates
# absentees, and can save the report to a file.

import re
from datetime import datetime

TIME_FORMAT = "%I:%M %p"  # example: 09:15 AM

def is_valid_time(tstr: str) -> bool:
    """Validate time string is in format HH:MM AM/PM (e.g. 09:15 AM)."""
    try:
        datetime.strptime(tstr.strip(), TIME_FORMAT)
        return True
    except Exception:
        return False

def get_nonempty_input(prompt: str) -> str:
    """Prompt until non-empty string returned."""
    while True:
        s = input(prompt).strip()
        if s:
            return s
        print("Name cannot be empty. Please enter a valid name.")

def get_valid_time_input(prompt: str) -> str:
    """Prompt until a valid timestamp is entered in the expected format."""
    while True:
        t = input(prompt).strip()
        if not t:
            print("Timestamp cannot be empty. Please re-enter time in format HH:MM AM/PM (e.g. 09:15 AM).")
            continue
        if is_valid_time(t):
            # Normalize to standard format (e.g. 9:5 AM -> 09:05 AM)
            dt = datetime.strptime(t, TIME_FORMAT)
            return dt.strftime(TIME_FORMAT)
        else:
            print("Invalid time format. Use HH:MM AM/PM (e.g. 09:15 AM). Try again.")

def print_summary(attendance: dict, total_strength: int | None = None):
    """Prints a neat attendance summary table."""
    print("\nAttendance Summary")
    print("-" * 40)
    header_name = "Student Name"
    header_time = "Check-in Time"
    print(f"{header_name:20s} {header_time}")
    print("-" * 40)
    for name, t in attendance.items():
        print(f"{name:20s} {t}")
    print("-" * 40)
    total_present = len(attendance)
    print(f"Total Students Present: {total_present}")
    if isinstance(total_strength, int):
        total_absent = max(0, total_strength - total_present)
        print(f"Total Students (Class Strength): {total_strength}")
        print(f"Total Absent: {total_absent}")

def save_report(attendance: dict, total_strength: int | None = None):
    """Save attendance to attendance_log.txt."""
    filename = "attendance_log.txt"
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, "a", encoding="utf-8") as f:
        f.write(f"Attendance Report - Generated on: {now}\n")
        f.write("-" * 60 + "\n")
        f.write(f"{'Student Name':30s} {'Check-in Time'}\n")
        f.write("-" * 60 + "\n")
        for name, t in attendance.items():
            f.write(f"{name:30s} {t}\n")
        f.write("-" * 60 + "\n")
        f.write(f"Total Present: {len(attendance)}\n")
        if isinstance(total_strength, int):
            f.write(f"Total Strength: {total_strength}\n")
            f.write(f"Total Absent: {max(0, total_strength - len(attendance))}\n")
        f.write("\n")
    print(f"Attendance saved to '{filename}'.")

def main():
    print("Welcome to the Command-Line Attendance Tracker")
    print("This tool records student names with check-in times, validates entries, and can save a report.\n")

    # number of entries to record
    while True:
        num_entries_raw = input("How many entries would you like to record? ").strip()
        if num_entries_raw.isdigit() and int(num_entries_raw) >= 0:
            num_entries = int(num_entries_raw)
            break
        print("Please enter a non-negative integer for the number of entries.")

    attendance = {}

    for i in range(num_entries):
        print(f"\nEntry {i+1} of {num_entries}:")
        # name input with validation (blank-check)
        name = get_nonempty_input("  Student name: ")
        # duplicate check
        if name in attendance:
            print("  This name is already recorded. Duplicate entries are not allowed. Skipping this entry.")
            continue
        # time input with validation
        time_str = get_valid_time_input("  Check-in time (HH:MM AM/PM, e.g. 09:15 AM): ")
        attendance[name] = time_str

    # Optional enhancement: ask total class strength for absentee calculation
    total_strength = None
    while True:
        resp = input("\nDo you want to provide total class strength to compute absentees? (y/n): ").strip().lower()
        if resp in ("y", "n"):
            break
        print("Please enter 'y' or 'n'.")
    if resp == "y":
        while True:
            ts_raw = input("Enter total number of students in the class: ").strip()
            if ts_raw.isdigit() and int(ts_raw) >= 0:
                total_strength = int(ts_raw)
                break
            print("Enter a non-negative integer for class strength.")

    # Print summary
    print_summary(attendance, total_strength)

    # Bonus: save to file
    while True:
        save_resp = input("\nDo you want to save the attendance report to file? (y/n): ").strip().lower()
        if save_resp in ("y", "n"):
            break
        print("Please enter 'y' or 'n'.")
    if save_resp == "y":
        save_report(attendance, total_strength)
    else:
        print("Report not saved. You can run the program again to save when ready.")

if __name__ == "__main__":
    main()


Welcome to the Command-Line Attendance Tracker
This tool records student names with check-in times, validates entries, and can save a report.

How many entries would you like to record? 1

Entry 1 of 1:
  Student name: Sam
  Check-in time (HH:MM AM/PM, e.g. 09:15 AM): 9:15
Invalid time format. Use HH:MM AM/PM (e.g. 09:15 AM). Try again.
  Check-in time (HH:MM AM/PM, e.g. 09:15 AM): AM
Invalid time format. Use HH:MM AM/PM (e.g. 09:15 AM). Try again.
  Check-in time (HH:MM AM/PM, e.g. 09:15 AM): 9:15 AM

Do you want to provide total class strength to compute absentees? (y/n): y
Enter total number of students in the class: 5

Attendance Summary
----------------------------------------
Student Name         Check-in Time
----------------------------------------
Sam                  09:15 AM
----------------------------------------
Total Students Present: 1
Total Students (Class Strength): 5
Total Absent: 4

Do you want to save the attendance report to file? (y/n): y
Attendance saved to 'att